In [1]:
import pandas as pd
import numpy as np
from copy import deepcopy
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_curve, precision_recall_curve, average_precision_score, f1_score
from sklearn.model_selection import GroupKFold, GroupShuffleSplit
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
import lightgbm as lgb
import xgboost as xgb
from catboost import Pool, CatBoostClassifier
from sklearn.model_selection import GridSearchCV

In [2]:
from IPython.core.display import clear_output
import sys
def change_output(x):
    clear_output()
    sys.stdout.write(x)
    sys.stdout.flush()


In [3]:
def load_data(filename, sep=","):
    '''
        Функция для загрузки датасета без инициализации класса. 
        
        Возвращает:
         data - полный DataFrame
         fetures - список признаков для обучения/предсказания
         x - датасет для тренировки/предсказания в формате np.array
         y - целевой признак, если он есть. В случае с тестовым датасетом возвращает None
    '''
    
    data = pd.read_table(filename, sep=sep).dropna()    
    
    data = data[~(data['chainlen']>1000)]

    if ('DSSR' in data.columns):
        data.drop('DSSR', axis=1, inplace=True)    
    
    features = list(deepcopy(data.columns))
    [features.remove(column) for column in ['Id','index', 'pdb_chain', 'mg'] if column in data.columns];
    x_test = np.array(data[features])
    
    try:
        y_test = np.array(data['mg'])
    except: 
        y_test = None
    change_output('Data loaded')
    return {'data':data, 'features':features, 'x':x_test, 'y':y_test}

In [4]:
test = load_data("test.csv")

Data loaded

In [5]:
change_output('Loading data...')        
data = pd.read_table("train.csv")
change_output('Data processing...')  
if ('DSSR' in data.columns):
    data.drop('DSSR', axis=1, inplace=True)            
data = data.dropna()      
y = deepcopy(np.array(np.matrix(data['mg']).flatten().tolist()[0])) 
data_numpy = np.matrix(data)
features = list(data.columns)
features.remove('pdb_chain')
features.remove('mg')
groups = np.array(data['pdb_chain'])
X = np.array(data[features])
features.append('mg')
change_output('Done')  

Done

In [6]:
X.shape

(209043, 383)

In [7]:
test["x"].shape

(4045, 383)

In [8]:
def xg_f1(y,t):
    t = t.get_label()
    y_bin = [1. if y_cont > 0.5 else 0. for y_cont in y] # binaryzing your output
    return 'f1',f1_score(t,y_bin)

In [9]:
def loglikelood(preds, train_data):
    labels = train_data  #.get_label()
    preds = 1. / (1. + np.exp(-preds))
    grad = preds - labels
    hess = preds * (1. - preds)
    return grad, hess

In [23]:
clf = lgb.LGBMClassifier(n_estimators=300, learning_rate=0.01, scale_pos_weight=2.0)
params = {
    "n_estimators":[300], #, 500, 1000, 5000],
    "learning_rate":[0.01], #, 0.05, 0.1, 0.5],
    "scale_pos_weight":np.linspace(0.1, 100, 50)
}

gscv = GridSearchCV(clf, params, scoring="f1", cv=5)
gscv.fit(X, y)

/home/shevkunov/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/shevkunov/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/shevkunov/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/shevkunov/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will

/home/shevkunov/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/shevkunov/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/shevkunov/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/shevkunov/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is a

/home/shevkunov/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/shevkunov/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/shevkunov/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/shevkunov/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is a

/home/shevkunov/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/shevkunov/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/shevkunov/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/shevkunov/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is a

/home/shevkunov/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/shevkunov/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/shevkunov/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/shevkunov/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is a

...

/home/shevkunov/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/shevkunov/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/shevkunov/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/shevkunov/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is a

/home/shevkunov/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/shevkunov/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/shevkunov/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/shevkunov/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is a

/home/shevkunov/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/shevkunov/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/shevkunov/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/shevkunov/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is a

/home/shevkunov/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/shevkunov/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/shevkunov/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/shevkunov/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is a

GridSearchCV(cv=5, error_score='raise',
       estimator=LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        learning_rate=0.01, max_depth=-1, min_child_samples=20,
        min_child_weight=0.001, min_split_gain=0.0, n_estimators=300,
        n_jobs=-1, num_leaves=31, objective=None, random_state=None,
        reg_alpha=0.0, reg_lambda=0.0, scale_pos_weight=2.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=1),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_estimators': [300], 'learning_rate': [0.01], 'scale_pos_weight': array([  0.1    ,   2.13878,   4.17755,   6.21633,   8.2551 ,  10.29388,
        12.33265,  14.37143,  16.4102 ,  18.44898,  20.48776,  22.52653,
        24.56531,  26.60408,  28.64286,  30.68163,  32.72041,  34.75918,
 ...     85.72857,  87.76735,  89.80612,  91.8449 ,  93.88367,  95.92245,
        97.96122, 100.     ])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='

In [24]:
gscv.grid_scores_

/home/shevkunov/.local/lib/python3.6/site-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.05323, std: 0.10577, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 0.1},
 mean: 0.06368, std: 0.09360, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 2.138775510204082},
 mean: 0.21756, std: 0.04924, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 4.177551020408163},
 mean: 0.28336, std: 0.03686, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 6.216326530612245},
 mean: 0.28418, std: 0.02498, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 8.255102040816327},
 mean: 0.27476, std: 0.01649, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 10.29387755102041},
 mean: 0.26686, std: 0.01281, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 12.332653061224491},
 mean: 0.26051, std: 0.00943, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 14.371428571428572},
 mean: 0.25459, std: 0.00911

In [25]:
gscv.best_score_

0.2841823044383953

In [26]:
gscv.best_params_

{'learning_rate': 0.01,
 'n_estimators': 300,
 'scale_pos_weight': 8.255102040816327}

In [33]:
# array([0.00772472, 0.00244328, 0.00176118, 0.238831  , 0.00281096]) RF 100
# array([0.1082544 , 0.11260222, 0.05920281, 0.26041424, 0.09496811]) DT 15
# array([0.0664723 , 0.07678245, 0.0174002 , 0.27320219, 0.03447715]) DT 10
# array([0.03578187, 0.01340897, 0.00581879, 0.00132802, 0.00586142]) CatBoost
# array([0.08434712, 0.05030018, 0.01561858, 0.00582165, 0.02612778]) CatBoost 3k
# array([0.10094637, 0.06313806, 0.02475649, 0.01292024, 0.03391199]) CatBoost 6k -> 0.18848
# ... CatBoost 9k -> 0.20512
# array([0.19595154, 0.19486361, 0.13402873, 0.24864054, 0.17646506]) DT -> 0.20076
# (array([0.15511116, 0.1459787 , 0.09896751, 0.25948915, 0.14122632]), 0.16015456743463227) DT md = 20
# (array([0.09605055, 0.11355198, 0.05333333, 0.07721804, 0.09993754]), 0.0880182881326547) ETC 
# (array([0.08023579, 0.10090881, 0.04891122, 0.06303872, 0.09434263]), 0.07748743541127794) ETC ne = 20

# (array([0.0952381 , 0.05911174, 0.01544269, 0.23348956, 0.04101367]), 0.08885915227716767) lgb ne 1000
# (array([0.08036001, 0.05293441, 0.0295868 , 0.23099507, 0.05625414]), 0.09002608666932135) lgb ne 10000 -> 0.19689
"""
[mean: 0.05323, std: 0.10577, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 0.1},
 mean: 0.06368, std: 0.09360, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 2.138775510204082},
 mean: 0.21756, std: 0.04924, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 4.177551020408163},
 mean: 0.28336, std: 0.03686, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 6.216326530612245},
 mean: 0.28418, std: 0.02498, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 8.255102040816327},
 mean: 0.27476, std: 0.01649, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 10.29387755102041},
 mean: 0.26686, std: 0.01281, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 12.332653061224491},
 mean: 0.26051, std: 0.00943, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 14.371428571428572},
 mean: 0.25459, std: 0.00911, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 16.410204081632656},
 mean: 0.25052, std: 0.00854, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 18.44897959183674},
 mean: 0.24771, std: 0.00776, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 20.487755102040822},
 mean: 0.24520, std: 0.00752, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 22.5265306122449},
 mean: 0.24357, std: 0.00746, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 24.565306122448984},
 mean: 0.24230, std: 0.00696, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 26.604081632653067},
 mean: 0.24141, std: 0.00649, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 28.642857142857146},
 mean: 0.24099, std: 0.00645, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 30.68163265306123},
 mean: 0.24033, std: 0.00659, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 32.72040816326531},
 mean: 0.24001, std: 0.00650, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 34.759183673469394},
 mean: 0.23971, std: 0.00622, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 36.79795918367348},
 mean: 0.23942, std: 0.00630, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 38.83673469387756},
 mean: 0.23914, std: 0.00604, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 40.87551020408164},
 mean: 0.23893, std: 0.00613, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 42.91428571428572},
 mean: 0.23895, std: 0.00575, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 44.9530612244898},
 mean: 0.23884, std: 0.00562, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 46.991836734693884},
 mean: 0.23861, std: 0.00563, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 49.030612244897966},
 mean: 0.23875, std: 0.00551, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 51.06938775510205},
 mean: 0.23881, std: 0.00554, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 53.10816326530613},
 mean: 0.23869, std: 0.00550, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 55.146938775510215},
 mean: 0.23872, std: 0.00552, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 57.18571428571429},
 mean: 0.23864, std: 0.00527, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 59.22448979591837},
 mean: 0.23866, std: 0.00518, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 61.263265306122456},
 mean: 0.23858, std: 0.00518, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 63.30204081632654},
 mean: 0.23858, std: 0.00511, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 65.34081632653061},
 mean: 0.23855, std: 0.00496, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 67.37959183673469},
 mean: 0.23852, std: 0.00509, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 69.41836734693878},
 mean: 0.23855, std: 0.00503, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 71.45714285714286},
 mean: 0.23844, std: 0.00495, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 73.49591836734695},
 mean: 0.23836, std: 0.00500, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 75.53469387755102},
 mean: 0.23841, std: 0.00496, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 77.57346938775511},
 mean: 0.23845, std: 0.00485, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 79.61224489795919},
 mean: 0.23834, std: 0.00495, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 81.65102040816328},
 mean: 0.23836, std: 0.00481, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 83.68979591836735},
 mean: 0.23830, std: 0.00479, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 85.72857142857143},
 mean: 0.23834, std: 0.00483, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 87.76734693877552},
 mean: 0.23836, std: 0.00479, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 89.8061224489796},
 mean: 0.23844, std: 0.00481, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 91.84489795918368},
 mean: 0.23827, std: 0.00483, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 93.88367346938776},
 mean: 0.23833, std: 0.00469, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 95.92244897959185},
 mean: 0.23839, std: 0.00475, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 97.96122448979592},
 mean: 0.23829, std: 0.00484, params: {'learning_rate': 0.01, 'n_estimators': 300, 'scale_pos_weight': 100.0}]
 
 
 gscv.best_score_

0.2841823044383953  -> 0.34377

gscv.best_params_

{'learning_rate': 0.01,
 'n_estimators': 300,
 'scale_pos_weight': 8.255102040816327}
 """

In [28]:
# clf = CatBoostClassifier(iterations=9000)
#clf = DecisionTreeClassifier(random_state=17)
best_params = gscv.best_params_
best_params["random_state"] = 17
clf = lgb.LGBMClassifier(**best_params)
clf.fit(X, y)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        learning_rate=0.01, max_depth=-1, min_child_samples=20,
        min_child_weight=0.001, min_split_gain=0.0, n_estimators=300,
        n_jobs=-1, num_leaves=31, objective=None, random_state=17,
        reg_alpha=0.0, reg_lambda=0.0, scale_pos_weight=8.255102040816327,
        silent=True, subsample=1.0, subsample_for_bin=200000,
        subsample_freq=1)

In [29]:
prediction = clf.predict(test["x"]).astype(int)

/home/shevkunov/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [30]:
np.unique(prediction, return_counts=True)

(array([0, 1]), array([2744, 1301]))

In [31]:
df_sub = pd.read_csv("sample_submission.csv")
df_sub["mg"] = prediction
df_sub.to_csv("my_sub.csv", index=False)

In [32]:
df_sub

,Id,mg
0,0,1
1,1,1
2,2,1
3,3,0
4,4,1
5,5,0
6,6,1
7,7,0
8,8,1
9,9,1
